<a href="https://colab.research.google.com/github/IverMartinsen/MastersThesis/blob/main/Notebooks/cod_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###This notebook shows the model training of a CNN fitted to the cod otolith data.
-----------------------------------------------------------------------------------

Clone GitHub repository

In [ ]:
!git clone https://github.com/IverMartinsen/MastersThesis.git

Import modules

In [ ]:
import sys
sys.path.append('/content/MastersThesis/Python')

import tensorflow as tf
import matplotlib.pyplot as plt
from modules.imageloader import load_images
from modules.utils import normalize
from modules.analysis.guided_backpropagation import build_gb_model_nonseq
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout,Dense
from tensorflow.keras.applications.xception import preprocess_input
import numpy as np

Import images and set variables


In [ ]:
img_size = (128, 128)
img_shape = img_size + (3,)
initial_epochs = 100
batch_size = 32
path_to_files = '/content/drive/MyDrive/Torskeotolitter/standard'

train_ds, valid_ds = load_images(
    path_to_files, img_size, splits=(0.8, 0.2), seed=123, mode='RGB'
    )

Build a new model

In [ ]:
base_model = tf.keras.applications.Xception(
    input_shape=img_shape,
    include_top=False,
    weights='imagenet'
    )

# Freeze layers in base model
for layer in base_model.layers:
    layer.trainable = False

inputs = tf.keras.Input(img_shape)
x = preprocess_input(inputs)
# Set training=False to freeze batch normalization weights
x = base_model(x, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
# Important! Use a single output with 'sigmoid' activation for normal training.
# If input is to be analyzed by Guided Backpropagation using a baseline class,
# set number of output units to 3 with 'softmax' activation.
outputs = Dense(1, 'sigmoid')(x)

model = tf.keras.Model(inputs, outputs)
# Note! Use binary cross-entropy for single outputs and categorical cross-entropy for
# multiple outputs.
model.compile(
    tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics='accuracy'
    )

Fit the new model

In [ ]:
callbacks = [tf.keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)]

# train only the top layer
history = model.fit(
    train_ds['images'], 
    train_ds['labels'],
    batch_size=batch_size,
    epochs=initial_epochs,
    callbacks=callbacks,
    validation_data=(valid_ds['images'], valid_ds['labels'])
)

# fine tune the model using a small learning rate
base_model.trainable = True

# Note! Use binary cross-entropy for single outputs and categorical cross-entropy for
# multiple outputs.
model.compile(    
    tf.keras.optimizers.Adam(1e-5),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics='accuracy')

history_fine = model.fit(
    train_ds['images'],
    train_ds['labels'],
    batch_size=batch_size,
    epochs=100,
    callbacks=callbacks,
    validation_data=(valid_ds['images'], valid_ds['labels'])
)

Save model

In [ ]:
model.save('/content/drive/MyDrive/Saved models/cod_model_' + tf.__version__)

Build Guided Backpropagation model using weights from the trained model

In [ ]:
gb_base_model = build_gb_model_nonseq(base_model, tf.keras.layers.Activation)

inputs = tf.keras.Input(img_shape)
x = preprocess_input(inputs)
x = gb_base_model(x, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
outputs = Dense(3, 'softmax')(x)

gb_model = tf.keras.Model(inputs, outputs)

gb_model.compile(    
    tf.keras.optimizers.Adam(1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics='accuracy')

# Load the weights from trained model
gb_model.set_weights(model.get_weights())

Save Guided Backpropagation model

In [ ]:
gb_model.save('/content/drive/MyDrive/Artikkel om torskeotolitter/Saved models/Xception_' + tf.__version__ + '_gb')